### Imports

In [ ]:
%pip install yfinance
%pip install matplotlib
%pip install scikit-learn
%pip install lxml

In [ ]:
import pandas as pd
import numpy as np
import pickle
import requests
import copy
import yfinance as yf
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats.mstats import winsorize
from sklearn.linear_model import LinearRegression

## Data collection

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}
html = requests.get(url, headers=headers)
SPY_tickers = pd.read_html(html.text)[0]['Symbol'].tolist()
SPY_tickers.append("^GSPC")

In [ ]:
data = yf.download(SPY_tickers, start="2000-01-01", end="2025-11-25", auto_adjust=True)

In [ ]:
data.to_parquet("data/SPY_data.pq")

## Dataset creation

In [ ]:
data = pd.read_parquet("data/SPY_data.pq")

In [ ]:
selected_stocks = data['Close'].columns.unique()[(data['Close'].isna().mean()<0.1)]

In [ ]:
len(selected_stocks)

In [ ]:
data.columns.get_level_values(0).unique()

In [ ]:
log_close = np.log(data['Close'])
log_open  = np.log(data['Open'])
log_high  = np.log(data['High'])
log_low   = np.log(data['Low'])

stocks = selected_stocks[:-1]
index = '^GSPC'

# --- returns ---
close_diff = log_close[stocks].diff()
index_ret  = log_close[index].diff()

df = close_diff.mul(1e4).stack().rename("close_1d_ret").reset_index()

df["close_1d_ret_hedged"] = (
    (close_diff.sub(index_ret, axis=0) * 1e4)
    .stack()
    .values
)

# --- open-close ---
open_close = (log_close[stocks] - log_open[stocks]) * 1e4
open_close_idx = (log_close[index] - log_open[index]) * 1e4

df = df.merge(
    open_close.stack().rename("open_close_ret").reset_index(),
    on=["Date", "Ticker"]
)

df = df.merge(
    (open_close.sub(open_close_idx, axis=0))
        .stack()
        .rename("open_close_ret_hedged")
        .reset_index(),
    on=["Date", "Ticker"]
)

# --- close-open ---

close_open = (log_open[stocks] - log_close[stocks].shift()) * 1e4
close_open_idx = (log_open[index] - log_close[index].shift()) * 1e4

df = df.merge(
    close_open.stack().rename("close_open_ret").reset_index(),
    on=["Date", "Ticker"]
)

df = df.merge(
    (close_open.sub(close_open_idx, axis=0))
        .stack()
        .rename("close_open_ret_hedged")
        .reset_index(),
    on=["Date", "Ticker"]
)

# --- high-low ---
high_low = (log_high[stocks] - log_low[stocks]) * 1e4
high_low_idx = (log_high[index] - log_low[index]) * 1e4

df = df.merge(
    high_low.stack().rename("high_low_ret").reset_index(),
    on=["Date", "Ticker"]
)

df = df.merge(
    (high_low.sub(high_low_idx, axis=0))
        .stack()
        .rename("high_low_ret_hedged")
        .reset_index(),
    on=["Date", "Ticker"]
)

# --- volumes ---
volume = data['Volume'][stocks]
dollar_volume = volume * (data['Open'][stocks] + data['Close'][stocks])/2
dolar_volume_share = volume.div(volume.sum(axis=1), axis=0)

df = df.merge(
    volume.stack().rename("volume").reset_index(),
    on=["Date", "Ticker"]
)

df = df.merge(
    dollar_volume.stack().rename("dollar_volume").reset_index(),
    on=["Date", "Ticker"]
)

df = df.merge(
    dolar_volume_share.stack().rename("share_dollar_volume").reset_index(),
    on=["Date", "Ticker"]
)

In [ ]:
df['body_ratio'] = df['open_close_ret'] / (df['high_low_ret']+1e-8)

In [ ]:
df['turnover_proxy'] = df['dollar_volume'] / df.groupby('Date')['dollar_volume'].transform('mean')

In [ ]:
df.set_index('Date', inplace=True)

In [ ]:
df['close_1d_ret_lag1'] = df.groupby('Ticker')['close_1d_ret'].shift()
df['close_1d_ret_hedged_lag1'] = df.groupby('Ticker')['close_1d_ret_hedged'].shift()
df['open_close_ret'] = df.groupby('Ticker')['open_close_ret'].shift()
df['open_close_ret_hedged'] = df.groupby('Ticker')['open_close_ret_hedged'].shift()
df['close_open_ret'] = df.groupby('Ticker')['close_open_ret'].shift()
df['close_open_ret_hedged'] = df.groupby('Ticker')['close_open_ret_hedged'].shift()
df['high_low_ret'] = df.groupby('Ticker')['high_low_ret'].shift()
df['high_low_ret_hedged'] = df.groupby('Ticker')['high_low_ret_hedged'].shift()
df['share_dollar_volume'] = df.groupby('Ticker')['share_dollar_volume'].shift()
df['body_ratio'] = df.groupby('Ticker')['body_ratio'].shift()
df['turnover_proxy'] = df.groupby('Ticker')['turnover_proxy'].shift()

In [ ]:
df.tail()

In [ ]:
linear_features = ['close_1d_ret_lag1', 'close_1d_ret_hedged_lag1', 'open_close_ret', 'open_close_ret_hedged', 'close_open_ret', 'close_open_ret_hedged', 'high_low_ret', 'high_low_ret_hedged', 'body_ratio']
non_linear_features = ['share_dollar_volume', 'turnover_proxy']

# Feature list
1. Avg past returns close to close
2. Avg past returns open to close
3. Avg past returns low to close
4. Avg past returns high to low

- Hedged/Not hedged
- Clipped/Not clipped

# Models list

## Baseline
1. Random
2. Past returns (define period)
3. MACD vol adjusted
4. Linear Regression (define features + beta)

## LTR
1. LambdaMART (pairwise)
2. LambdaRANK (listwise)
3. ListMLE (listwise - use LightGBM)

# Baseline models

## Random Strategy

In [ ]:
np.random.seed(42)
df['random_signal'] = 0.0

def assign_random_signals(group):
    n = len(group)
    if n >= 70:
        signals = np.array([1.0] * 35 + [-1.0] * 35 + [0.0] * (n - 70))
        np.random.shuffle(signals)
        return pd.Series(signals, index=group.index)
    else:
        return pd.Series(0.0, index=group.index)

df['random_signal'] = df.groupby('Date', group_keys=False).apply(assign_random_signals)

In [ ]:
random_daily_returns = df.groupby('Date').apply(lambda x: (x['random_signal']*x['close_1d_ret']).mean())

In [ ]:
(random_daily_returns.cumsum()*1e-4+1).plot(figsize=(12,6), title='Cumulative Random Strategy Return', grid=True)

In [ ]:
random_daily_returns.mean()/random_daily_returns.std()*np.sqrt(252)

## Momentum Strategies

### Simple Momentum Strategy

In [ ]:
def compute_signal(x):
    if x.notna().sum() == 0:
        return pd.Series([np.nan]*len(x), index=x.index)
    ranks = x.rank(method='first')
    binned = pd.cut(ranks, bins=10, labels=False, include_lowest=True)
    signal = binned.map(lambda y: 1 if y == 0 else (-1 if y == 9 else 0))
    return signal

In [ ]:
df['momentum_hedged_signal'] = df.groupby('Date')['close_1d_ret_hedged'].transform(compute_signal)

In [ ]:
df['momentum_hedged_signal'] = df.groupby('Ticker')['momentum_hedged_signal'].shift()

In [ ]:
df['momentum_signal'] = df.groupby('Date')['close_1d_ret'].transform(compute_signal)

In [ ]:
df['momentum_signal'] = df.groupby('Ticker')['momentum_signal'].shift()

In [ ]:
momentum_daily_returns = df.groupby('Date').apply(
    lambda x: pd.Series({
        'hedged': (x['momentum_hedged_signal'] * x['close_1d_ret']).mean(),
        'unhedged': (x['momentum_signal'] * x['close_1d_ret']).mean()
    })
)*1e-4

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(momentum_daily_returns['hedged'].cumsum() + 1, label='Hedged Momentum Strategy')
plt.plot(momentum_daily_returns['unhedged'].cumsum() + 1, label='Unhedged Momentum Strategy')

plt.title('Cumulative Momentum Strategy Returns')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
momentum_daily_returns.mean()/momentum_daily_returns.std()*np.sqrt(252)

### MACD Vol adjusted Strategy

In [ ]:
short_span_list = [8, 16, 32]
long_span_list = [24, 48, 96]

In [ ]:
for i, (short_span, long_span) in enumerate(zip(short_span_list, long_span_list)):
    macd_adj_series = (df.groupby('Ticker')['close_1d_ret'].ewm(span=short_span, adjust=False).mean() - df.groupby('Ticker')['close_1d_ret'].ewm(span=long_span, adjust=False).mean())/df.groupby('Ticker')['close_1d_ret'].rolling(63).std()
    macd_adj_series.name = f'macd_adj_{i+1}'
    df = df.merge(macd_adj_series, on=['Date', 'Ticker'], how='right')
    df[f'macd_adj_{i+1}'] /= df[f'macd_adj_{i+1}'].rolling(252).std()

In [ ]:
def phi_baz(x):
    return x / np.sqrt(1 + x**2)

In [ ]:
df['macd_baz_signal'] = df[['macd_adj_1', 'macd_adj_2', 'macd_adj_3']].apply(phi_baz).sum(1).replace(0, np.nan)
df['macd_baz_signal'] = df.groupby('Date')['macd_baz_signal'].transform(compute_signal)

In [ ]:
df['macd_tanh_signal'] = df[['macd_adj_1', 'macd_adj_2', 'macd_adj_3']].apply(np.tanh).sum(1).replace(0, np.nan)
df['macd_tanh_signal']  = df.groupby('Date')['macd_tanh_signal'].transform(compute_signal)

In [ ]:
df['macd_baz_signal'] = df.groupby('Ticker')['macd_baz_signal'].shift()
df['macd_tanh_signal'] = df.groupby('Ticker')['macd_tanh_signal'].shift()

In [ ]:
macd_daily_returns = df.groupby('Date').apply(
    lambda x: pd.Series({
        'baz': (x['macd_baz_signal'] * x['close_1d_ret']).mean(),
        'tanh': (x['macd_tanh_signal'] * x['close_1d_ret']).mean()
    })
)*1e-4

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(macd_daily_returns['baz'].cumsum() + 1, label='MACD Strategy - Baz function')
plt.plot(macd_daily_returns['tanh'].cumsum() + 1, label='MACD Strategy - Tanh function')

plt.title('Cumulative Momentum Strategy Returns')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
macd_daily_returns.mean()/macd_daily_returns.std()*np.sqrt(252)

# Regress-then-rank Strategies 

## Linear Regression

### Feature factory

In [ ]:
def daily_metrics(group, feature, target, beta_global):
    reg = LinearRegression(fit_intercept=False)
    if len(group) < 2:
        return pd.Series({'beta': np.nan, 'bias': np.nan})
    reg.fit(group[[feature]], group[target])
    predictions = reg.predict(group[[feature]])
    bias = np.std(predictions)
    if reg.coef_[0]*beta_global < 0:
        bias *= -1
    return pd.Series({'bias': bias})

In [ ]:
def single_feature_metrics(df, feature, target, drop_extreme_perc=True, ts_bool=False, fit_intercept=False):
    global_reg = LinearRegression(fit_intercept=fit_intercept)
    if drop_extreme_perc:
        first_perc, last_perc = np.percentile(df[feature].dropna(), [1, 99])
        df_filtered = df[(df[feature]>=first_perc) & (df[feature]<=last_perc)]
    else:
        df_filtered = df.copy(deep=True)
    global_reg.fit(df_filtered[[feature]], df_filtered[target])
    predictions = global_reg.predict(df_filtered[[feature]])
    bias = np.std(predictions)
    stab = (predictions * df_filtered[target] > 0).mean() * 100
    beta_global = global_reg.coef_[0]
    if ts_bool:
        bias_ts = df_filtered.groupby('Date').apply(lambda x: daily_metrics(x, feature, target, beta_global))
        mean_bias_ts = bias_ts['bias'].mean()
        sharpe_ts = bias_ts['bias'].mean() / bias_ts['bias'].std() * np.sqrt(252)
        return bias, mean_bias_ts, stab, beta_global, sharpe_ts, bias_ts
    else:
        return bias, stab, beta_global

In [ ]:
linear_features

In [ ]:
all_linear_features = copy.deepcopy(linear_features)

In [ ]:
for feature in linear_features:
    print(feature)
    plt.hist(pd.Series(winsorize(df[feature], limits=0.001)).dropna(), bins=100)
    plt.title(f'{feature} hist')
    plt.show()

In [ ]:
for feature in tqdm(linear_features):
    for h in [3, 5, 10, 15, 20, 30, 60, 80]:
        df[f'avg_{feature}_{h}d'] = df.groupby('Ticker')[feature].transform(lambda x: x.rolling(h).mean())
        df[f'zscore_{feature}_{h}d'] = (df[feature]-df[f'avg_{feature}_{h}d'])/df.groupby('Ticker')[feature].transform(lambda x: x.rolling(h).std())
        df[f'sharpe_{feature}_{h}d'] = df[f'avg_{feature}_{h}d']/df.groupby('Ticker')[feature].transform(lambda x: x.rolling(h).std())
        all_linear_features.append(f'avg_{feature}_{h}d')
        all_linear_features.append(f'zscore_{feature}_{h}d')
        all_linear_features.append(f'sharpe_{feature}_{h}d')

In [ ]:
df.to_parquet('df.pq')

In [ ]:
for feature in tqdm(all_linear_features):
    df[feature] = winsorize(df[feature], limits=0.001)

In [ ]:
# for feature in linear_features:
#     print(feature)
#     plt.hist(df[feature].replace([-np.inf, np.inf], np.nan), bins=100)
#     plt.title(f'{feature} hist')
#     plt.show()
#     for feature_type in ['avg', 'zscore', 'sharpe']:
#         for window in [5, 30, 80]:
#             temp_feature = f'{feature_type}_{feature}_{window}d'
#             plt.hist(df[temp_feature].replace([-np.inf, np.inf], np.nan), bins=100)
#             plt.title(f'{temp_feature} hist')
#             plt.show()

In [ ]:
dict_results = {'feature': [], 'bias': [], 'mean_bias_ts': [], 'stability': [], 'beta': [], 'sharpe_ts': [], 'bias_ts': []}
for feature in tqdm(all_linear_features):
    bias, mean_bias_ts, stability, beta, sharpe_ts, bias_ts = single_feature_metrics(df[[feature, 'close_1d_ret']].dropna(), feature, 'close_1d_ret', drop_extreme_perc=False,  ts_bool=True)
    dict_results['feature'].append(feature)
    dict_results['bias'].append(bias)
    dict_results['mean_bias_ts'].append(mean_bias_ts)
    dict_results['stability'].append(stability)
    dict_results['beta'].append(beta)
    dict_results['sharpe_ts'].append(sharpe_ts)
    dict_results['bias_ts'].append(bias_ts)

In [ ]:
# with open("dict_results.pkl", "wb") as f:
#     pickle.dump(dict_results, f)

with open("dict_results.pkl", "rb") as f:
    dict_results = pickle.load(f)

In [ ]:
results_df = pd.DataFrame(dict_results).set_index('feature')
results_df['feature_type'] = pd.Series(results_df.index.str.split('_')).apply(lambda x: x[0] if x[0] in ['avg', 'zscore', 'sharpe'] else 'spot').values
results_df['window'] = pd.Series(results_df.index.str.split('_')).apply(lambda x: int(x[-1][:-1]) if (x[-1][-1]=='d' and x[-1]!='hedged') else 1).values
results_df['feature_name'] = np.where(results_df['feature_type']!='spot', pd.Series(results_df.index.str.split('_')).apply(lambda x: '_'.join(x[1:-1])), results_df.index)

In [ ]:
linear_features

In [ ]:
results_df.sort_values('mean_bias_ts').tail(50)

In [ ]:
results_df.loc['zscore_open_close_ret_60d', 'bias_ts'].cumsum().plot()

### Ridge regression

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
ridge_reg = Ridge(fit_intercept=False)

In [ ]:
df = pd.read_parquet('df.pq')

In [ ]:
df.head()

In [ ]:
X = df[all_linear_features + ["close_1d_ret"]].replace([-np.inf, np.inf], np.nan).dropna()[all_linear_features]
y = df[all_linear_features + ["close_1d_ret"]].replace([-np.inf, np.inf], np.nan).dropna()["close_1d_ret"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=False
)

# ---------------------------
# Expanding CV function
# ---------------------------
def expanding_cv_score(model, X, y, n_splits=5, lower=0.01, upper=0.01):
    """
    Performs expanding-window CV with winsorization applied to X.
    Winsorization limits are fit on each training window and applied to validation window.
    
    Parameters:
    - model: estimator with fit/predict
    - X: pandas DataFrame
    - y: pandas Series
    - n_splits: number of expanding CV splits
    - lower, upper: winsorization proportions
    """
    n_samples = len(X)
    split_size = (n_samples // 2) // n_splits  # size of each incremental step

    mse_list = []

    for i in tqdm(range(n_splits)):
        train_end = n_samples//2 + split_size * i
        val_end = n_samples//2 + split_size * (i + 1)

        # Slice windows
        X_train_cv = X.iloc[:train_end].copy()
        y_train_cv = y.iloc[:train_end]

        X_val_cv = X.iloc[train_end:val_end].copy()
        y_val_cv = y.iloc[train_end:val_end]

        # ---- Winsorization ----
        # Fit limits on TRAINING data only
        lower_bounds = {}
        upper_bounds = {}

        for col in X.columns:
            lb = np.percentile(X_train_cv[col], lower * 100)
            ub = np.percentile(X_train_cv[col], 100 - upper * 100)

            lower_bounds[col] = lb
            upper_bounds[col] = ub

            # apply to train
            X_train_cv[col] = np.clip(X_train_cv[col], lb, ub)
            # apply SAME limits to val
            X_val_cv[col] = np.clip(X_val_cv[col], lb, ub)

        # Fit + evaluate model
        model.fit(X_train_cv, y_train_cv)
        y_pred_cv = model.predict(X_val_cv)
        mse_list.append(mean_squared_error(y_val_cv, y_pred_cv))

    return mse_list

In [ ]:
# ---------------------------
# Hyperparameter tuning
# ---------------------------
alphas = [0.1, 0.3, 1.0, 3.0, 10.0, 30.0, 100.0]
alpha_mse = {}

for a in tqdm(alphas):
    ridge = Ridge(alpha=a, fit_intercept=False)
    mse = expanding_cv_score(ridge, X_train, y_train, n_splits=5)
    alpha_mse[a] = mse

In [ ]:
# ---------------------------
# Final model training on full training set
# ---------------------------
ridge_reg = Ridge(alpha=best_alpha, fit_intercept=False)
ridge_reg.fit(X_train, y_train)

y_pred = ridge_reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print("\nRidge Coefficients:", ridge_reg.coef_)
print("Ridge Intercept:", ridge_reg.intercept_)
print("Test MSE:", mse)

In [ ]:
for h in [3, 5, 10, 20, 50, 100]:
    for clip in [100, 200, 300, 400]:
        df[f'avg_ret_hedged_{h}d_clip{clip}'] = df[f'avg_ret_hedged_{h}d'].clip(-clip, clip)

In [ ]:
plt.figure(figsize=(12, 6))
for h in dict_results['avg_ret_hedged'].keys():
    dict_results['avg_ret_hedged'][h][-1]['bias'].cumsum().plot(label=f'{h} days')
plt.legend()
plt.title('Cumulative Bias Over Time')
plt.xlabel('Date')
plt.ylabel('Cumulative Bias')
plt.grid(True, alpha=0.3)
plt.show()